<center><img src="../Picture Data/logo.png" alt="Header" style="width: 800px;"/></center>

In [1]:
import ipywidgets.widgets as widgets
controller = widgets.Controller(index=0)  #Replace with the index number of the controller you just tested
display(controller)

Controller()

Import some module

In [2]:
#Import function library path
from servoserial import ServoSerial
from jetbot import Robot
import traitlets
from jetbot import Camera
from jetbot import bgr8_to_jpeg
from jetbot import Heartbeat
import threading
import time
# Thread function operation library
import inspect
import ctypes

import traitlets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
import os
from uuid import uuid1
import cv2
import numpy as np
import PID
from RGB_Lib import Programing_RGB
RGB = Programing_RGB()

In [3]:
global color_x, color_y, color_radius
color_x = color_y = color_radius = 0
global color_lower
global color_upper

Create and open a display camera initialization instance

In [4]:
camera = Camera.instance(width=300, height=300)
color_image = widgets.Image(format='jpeg', width=300, height=300)
camera_link = traitlets.dlink((camera, 'value'), (color_image, 'value'), transform=bgr8_to_jpeg)

Add heartbeat connection

In [5]:
def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        camera_link.unlink()
        robot.stop()

heartbeat = Heartbeat(period=0.5)
# Append the callback function to the heartbeat state
heartbeat.observe(handle_heartbeat_status, names='status')

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/jetbot-0.3.0-py3.6.egg/jetbot/heartbeat.py", line 35, in _run
    self.status = Heartbeat.Status.dead
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 588, in __set__
    self.set(obj, value)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 577, in set
    obj._notify_trait(self.name, old_value, new_value)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 1210, in _notify_trait
    type='change',
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 1215, in notify_change
    return self._notify_observers(change)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py",

Create a method to actively stop the process

In [6]:
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

Method for creating up, down, left, and right movements of a PTZ camera separately

In [7]:
'''Servo Part'''
servo_device = ServoSerial() 
def camUpFunction():
    global updownpulse
    updownpulse+=15
    if updownpulse>4095:
        updownpulse=4095
    servo_device.Servo_serial_control(2, updownpulse)

def camDownFunction():
    global updownpulse
    updownpulse-=15
    if updownpulse<1300:
        updownpulse=1300
    servo_device.Servo_serial_control(2, updownpulse)

def camLeftFunction():
    global leftrightpulse
    leftrightpulse+=15
    if leftrightpulse>3600:
        leftrightpulse=3600
    servo_device.Servo_serial_control(1, leftrightpulse)

def camRightFunction():
    global leftrightpulse
    leftrightpulse-=15
    if leftrightpulse<600:
        leftrightpulse=600
    servo_device.Servo_serial_control(1, leftrightpulse)

def camservoInitFunction():
    SERVO_MID = 2180
    global leftrightpulse, updownpulse
    leftrightpulse = 2048
    updownpulse = 2048
    servo_device.Servo_serial_control(1, SERVO_MID)
    time.sleep(0.1)
    servo_device.Servo_serial_control(2, 2048)

serial Open!


### Load the Robot class
This class allows us to easily control the JetBot motor

In [8]:
robot = Robot()

In [27]:
def BLN_Onboard():
    count4 = 0
    thread2 = threading.Thread(target=jetbot_motion)
    thread2.setDaemon(True)
    thread3 = threading.Thread(target=color_detect)
    thread3.setDaemon(True)
    thread4 = threading.Thread(target=auto_drive)
    thread4.setDaemon(True)
    global i , k
    i = k = 0
    while True:
        if k == 0:
            robot.set_bln(i)
            i += 0.01
            if( i >= 1 ):
                k = 1
            time.sleep(0.005)
        elif k == 1:
            robot.set_bln(i)
            i -=0.01
            if i <= 0 :
                k = 0
            time.sleep(0.005)
        if controller.buttons[0].value == True:
            count4 += 1
            if count4 >= 3:
                if thread2.isAlive() == False:
                    thread2 = threading.Thread(target=jetbot_motion)
                    thread2.start()
                if thread3.isAlive():
                    stop_thread(thread3)
                if thread4.isAlive():
                    stop_thread(thread4)
                    robot.stop()
                count4 = 0
                
        elif controller.buttons[1].value == True:
            count4 += 1
            if count4 >= 3:
                if thread3.isAlive() == False:
                    thread3 = threading.Thread(target=color_detect)
                    thread3.start()
                if thread4.isAlive():
                    stop_thread(thread4)
                    robot.stop()
                count4 = 0
                
        elif controller.buttons[2].value == True:
            count4 += 1
            if count4 >= 3:
                if thread2.isAlive():
                    stop_thread(thread2)
                if thread3.isAlive() == False:
                    stop_thread(thread4)
                    robot.stop()       
                if thread4.isAlive():
                    thread4 = threading.Thread(target=auto_drive)
                    thread4.start()
                count4 = 0
                
        elif controller.buttons[3].value == True:
            count4 += 1
            if count4 >= 3:
                if thread2.isAlive():
                    stop_thread(thread2)
                if thread3.isAlive():
                    stop_thread(thread3)
                if thread4.isAlive():
                    stop_thread(thread4)
                robot.stop()
                count4 = 0
        else:
            count4 = 0

Turn on the onboard breathing light independent process by running the cell code below

In [28]:
thread1 = threading.Thread(target=BLN_Onboard)
thread1.start()

In [25]:
stop_thread(thread1)

In [32]:
display(color_image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [10]:
def jetbot_motion():
    count1 = count2 = count3 = count4 =  count5 = 0
    while 1:
        
        if controller.axes[1].value <= 0.1:
            if (controller.axes[0].value <= 0.1 and controller.axes[0].value >= -0.1 
                and controller.axes[1].value <= 0.1 and controller.axes[1].value >= -0.1):
                robot.stop()
            else:
                robot.set_motors(-controller.axes[1].value + controller.axes[0].value, -controller.axes[1].value - controller.axes[0].value)
            
            time.sleep(0.01)
        else:
            robot.set_motors(-controller.axes[1].value - controller.axes[0].value, -controller.axes[1].value + controller.axes[0].value)
            time.sleep(0.01)
            
        if controller.axes[3].value == 1:
            count1  += 1
            if count1 >= 3:
                camDownFunction()
                count1 = 0
        elif controller.axes[3].value == -1:
            count1  += 1
            if count1 >= 3:
                camUpFunction()
                count1 = 0
        else:
            count1 = 0
        if controller.axes[2].value == 1:
            count2  += 1
            if count2 >= 3:
                camRightFunction()
                count2 = 0
        elif controller.axes[2].value == -1:
            count2  += 1
            if count2 >= 3:
                camLeftFunction()
                count2 = 0
        else:
            count2 = 0
        #Servo control camera up down, left and right is reset
        if controller.buttons[8].value == True:
            count3 += 1
            if count3 >= 3:
                camservoInitFunction()
                count3 = 0
        else:
            count3 = 0
        
        #Servo control servo rise and decline
        if controller.buttons[6].value == True:
            print()
            count4 += 1
            if count4 >= 3:
                robot.set_vertical_motors(1)
                count4 = 0
        elif controller.buttons[4].value == True:
            count4 += 1
            if count4 >= 3:
                robot.set_vertical_motors(-1)
                count4 = 0
        else:
            robot.set_vertical_motors(0)
            count4 = 0

In [11]:
def kmeans(X, k, max_iters=50):
    # 데이터 포인트의 수와 특성의 수 확인
    m, n = X.shape

    # 초기 중심 선택
    centroids = X[np.random.choice(m, k, replace=False)]

    for _ in range(max_iters):
       
        distances = np.linalg.norm(X[:, np.newaxis] - centroids, axis=2)
        labels = np.argmin(distances, axis=1)

        # 중심 재계산
        new_centroids = np.array([X[labels == j].mean(axis=0) for j in range(k)])

        # 수렴 여부 확인
        if np.all(centroids == new_centroids):
            break

        centroids = new_centroids

    return centroids, labels

In [16]:
def color_detect(): 
    RGB.OFF_ALL_RGB()
    frame = camera.value
    frame_c = frame.reshape(-1,3)
    data = frame_c
    k = 5
    centroids, labels = kmeans(data, k)

    labels, counts = np.unique(labels, return_counts=True)
    cen = labels[counts.argmax()]
    mask_color = centroids[cen].astype(int)
    
    mask_hsv = cv2.cvtColor(np.uint8([[mask_color]]),cv2.COLOR_BGR2HSV)
    mask_hsv = mask_hsv[0][0]
    color_lower = np.array([max(mask_hsv[0]-20,0),max(mask_hsv[1]-50,0),max(mask_hsv[2]-60,0)])
    color_upper = np.array([min(mask_hsv[0]+20,255),min(mask_hsv[1]+50,255),min(mask_hsv[2]+60,255)])
    RGB.Set_All_RGB(255,255,255)
    

In [21]:
def auto_drive():
    RGB.Set_All_RGB(0,255,0)
    while 1:
        frame = camera.value
        frame = cv2.resize(frame, (300, 300))
        frame_=cv2.GaussianBlur(frame,(5,5),0)                    
        hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        mask=cv2.inRange(hsv,color_lower,color_upper)  
        mask=cv2.erode(mask,None,iterations=2)
        mask=cv2.dilate(mask,None,iterations=2)
        mask=cv2.GaussianBlur(mask,(3,3),0)     
        cnts=cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2] 
        if len(cnts)>0:
            cnt = max (cnts,key=cv2.contourArea)
            (color_x,color_y),color_radius=cv2.minEnclosingCircle(cnt)
        if color_radius > 10:
            # Mark the detected color
            cv2.circle(frame,(int(color_x),int(color_y)),int(color_radius),(255,0,255),2)  
            #Proportion-Integration-Differentiation
            xservo_pid.SystemOutput = color_x


            target_x = 150
            target_y = 150
            speed = 0.8
            turn = np.abs(target_x-color_x)/300
            if color_x < target_x - 60:
                robot.set_motors(speed-turn, speed)
            elif color_x > target_x + 60:
                robot.set_motors(speed, speed-turn)
            elif color_radius>50:
                robot.stop()
            else:
                robot.set_motors(speed, speed)
    # Real-time return of image data for display"""
        color_image.value = bgr8_to_jpeg(frame)

Control the independent process of Jetbot motion in real time by running the cell code below to open the handle

In [57]:
thread2.start()

In [39]:
thread2.isAlive()

False

In [40]:
thread3.isAlive()

False

In [41]:
thread3 = threading.Thread(target=jetbot_color)
thread3.setDaemon(True)
thread3.start()

NameError: name 'jetbot_color' is not defined

Ending onboard breathing RGB process

In [64]:
stop_thread(thread2)

Ending Jetbot movement process

In [ ]:
stop_thread(thread2)
robot.stop()

In [ ]:
camera.stop()